In [ ]:
import numpy as np
import pandas as pd
import tensorboard as tf

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=749eb94f60932286c1e736780deffb03e01c401786883c5c8ade8cf70fc48bbb
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=84027ac50ccfb7130bc475e5c8f2f3ee034573475f12511590392df0b79791ce
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=3496d1c87d327da5512506516aec3addc993b8db66bf4a004abebee8bc4ee2f7
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 7.4MB/s 


In [ ]:
import bert
import tensorflow_hub as hub

In [ ]:
tokenizer=bert.bert_tokenization.FullTokenizer
bert_layer=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2',trainable=False)


In [ ]:
vocab=bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

In [ ]:
df=pd.read_csv("/content/drive/My Drive/train.csv")

In [ ]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
df.shape

(27481, 4)

In [ ]:
df.drop("textID",axis=1,inplace=True)
df.dropna(inplace=True)

In [ ]:
token=tokenizer(vocab_file=vocab,do_lower_case=do_lower_case)

In [ ]:
def get_tokens(sent,sentiment):
  return ['[CLS]']+token.tokenize(sent)+['[SEP]']+['[SEP]']+token.tokenize(sentiment)+['[SEP]']
def get_ids(tokens):
  return token.convert_tokens_to_ids(tokens)
def get_mask(sent,max_len):
  return [1]*len(sent)+[0]*(max_len-len(sent))
  


In [ ]:
def get_segments(tokens_list,seq):
  if seq%2==0:
    seq=0
  else:
    seq=1
  sent_rel=[]
  return [seq]*len(tokens_list)
        



In [ ]:
data=[get_tokens(sent,df.iloc[i,-1]) for i,sent in enumerate(df['text'])]

In [ ]:
max_len=max(len(sent) for sent in data)

In [ ]:
def get_pad(sent,max_len):
  temp_len=len(sent)
  req_len=max_len-temp_len
  final_sent=sent+['[PAD]']*req_len
  return get_ids(final_sent),final_sent

In [ ]:
a=get_pad(data[0],max_len)[1]
get_mask(data[0],max_len)


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
def get_outputs(train,test,max_len):
  out_1=[0]*max_len
  out_2=[0]*max_len
  test=token.tokenize(test)
  word_1=test[0]
  word_2=test[-1]
  for i in range(max_len):
    if train[i]==word_1:
      out_1[i]=1
      break
  for i in reversed(range(max_len)):
    if train[i]==word_2:
      out_2[i]=1
      break
  return out_1,out_2

In [ ]:
simple_input=[get_pad(sent,max_len)[0] for sent in data]

In [ ]:
input_mask=[get_mask(sent,max_len) for sent in data]

In [ ]:
sent_input=[get_segments(id_sent,i) for i,id_sent in enumerate(simple_input)]

In [ ]:

start_out_tok=[]
end_out_tok=[]
for i in range(df.shape[0]):
  try:
    a=get_pad(data[i],max_len)[1]
    o1=get_outputs(a,df.iloc[i,1],max_len)[0]
    o2=get_outputs(a,df.iloc[i,1],max_len)[1]
    start_out_tok.append(o1)
    end_out_tok.append(o2)
  except:
    print(f"the index{i}")
    break

In [ ]:
len(simple_input)==len(input_mask)==len(sent_input)==len(start_out_tok)==len(end_out_tok)

True

In [ ]:
simple_input=np.array(simple_input)
input_mask=np.array(input_mask)
sent_input=np.array(sent_input)
start_out_tok=np.array(start_out_tok)
end_out_tok=np.array(end_out_tok)

In [ ]:
split_size=int(0.75*len(simple_input))
simple_input_train=simple_input[:split_size,:]
input_mask_train=input_mask[:split_size,:]
sent_input_train=sent_input[:split_size,:]
start_out_tok_train=start_out_tok[:split_size,:]
end_out_tok_train=end_out_tok[:split_size,:]


In [ ]:
simple_input_valid=simple_input[split_size:,:]
input_mask_valid=input_mask[split_size:,:]
sent_input_valid=sent_input[split_size:,:]
start_out_tok_valid=start_out_tok[split_size:,:]
end_out_tok_valid=end_out_tok[split_size:,:]


In [ ]:
simple_input_train.shape==input_mask_train.shape==sent_input_train.shape

True

In [ ]:
simple_input_train.shape[0]==start_out_tok_train.shape[0]

True

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Conv1D,concatenate,Dropout,Softmax,Input,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

In [ ]:
input_1=Input((max_len,),dtype=tf.int32, name='input_word_ids')
input_2=Input((max_len,),dtype=tf.int32, name='input_mask')
input_3=Input((max_len,),dtype=tf.int32, name='input_type_ids')
bert_,_=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2',trainable=False)([input_1,input_2,input_3])
x=Conv1D(1,1)(_)
x=Flatten()(x)
outs1=Dense(max_len,activation='softmax')(x)
outs2=Dense(max_len,activation='softmax')(x)



In [ ]:
model=Model(inputs={'input_word_ids':input_1,'input_mask':input_2,'input_type_ids':input_3},outputs={'output1':outs1,'output2':outs2})

In [ ]:
inputs_bert={'input_word_ids':simple_input_train,'input_mask':input_mask_train,'input_type_ids':sent_input_train}
valid_bert={'input_word_ids':simple_input_valid,'input_mask':input_mask_valid,'input_type_ids':sent_input_valid}

In [ ]:
def custom_metric(s1,s2):
  in1=set(s1)
  in2=set(s2)
  intersec=in1.intersection(in2)
  return (intersec)/(len(in1)+len(in2)-intersec)

In [ ]:
import tensorflow as tf

In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy')

In [ ]:

outs={'output1':start_out_tok_train,'output2':end_out_tok_train}
outs_valid={'output1':start_out_tok_valid,'output2':end_out_tok_valid}
model.fit(inputs_bert,outs,epochs=10,validation_data=(valid_bert,outs_valid))

Epoch 1/10
645/645 [==============================] - 130s 202ms/step - loss: 0.2364 - dense_loss: 0.1094 - dense_1_loss: 0.1270 - val_loss: 0.2360 - val_dense_loss: 0.1097 - val_dense_1_loss: 0.1263
Epoch 2/10
645/645 [==============================] - 130s 202ms/step - loss: 0.2322 - dense_loss: 0.1094 - dense_1_loss: 0.1228 - val_loss: 0.2319 - val_dense_loss: 0.1097 - val_dense_1_loss: 0.1222
Epoch 3/10
645/645 [==============================] - 130s 201ms/step - loss: 0.2308 - dense_loss: 0.1094 - dense_1_loss: 0.1214 - val_loss: 0.2305 - val_dense_loss: 0.1097 - val_dense_1_loss: 0.1208
Epoch 4/10
645/645 [==============================] - 130s 202ms/step - loss: 0.2281 - dense_loss: 0.1094 - dense_1_loss: 0.1187 - val_loss: 0.2292 - val_dense_loss: 0.1097 - val_dense_1_loss: 0.1196
Epoch 5/10
645/645 [==============================] - 130s 202ms/step - loss: 0.2269 - dense_loss: 0.1093 - dense_1_loss: 0.1176 - val_loss: 0.2274 - val_dense_loss: 0.1097 - val_dense_1_loss: 0.1177


In [ ]:
tweet='finally got to see iphone it may not be revolutionary but absolutely gorgeous'
sentiment='positive'



In [ ]:
data_test=[get_tokens(tweet,sentiment)]

In [ ]:
test_input=[get_pad(t,max_len)[0] for t in data_test]
test_mask=[get_mask(t,max_len) for t in data_test]
test_sent_segment=[get_segments(x,i) for i,x in enumerate(test_input)]

In [ ]:
test_input=np.array(test_input)
test_mask=np.array(test_mask)
test_sent_segment=np.array(test_sent_segment)

In [ ]:
predictions=model.predict({'input_word_ids':test_input,'input_mask':test_mask,'input_type_ids':test_sent_segment})

In [ ]:
predictions.get('output1').shape

(1, 113)

In [ ]:
predictions.get('output2').shape

(1, 113)

In [ ]:
np.argmax(predictions.get('output1'),axis=-1)

array([12])

In [ ]:
np.argmax(predictions.get('output2'),axis=-1)

array([13])

In [ ]:
token_tweet=np.array([get_pad(t,max_len)[1] for t in data_test])

In [ ]:
print(f'the word_tokens responsible for sentiment are {token_tweet[:,12:14]}')

the word_tokens responsible for sentiment are [['absolutely' 'gorgeous']]
